In [1]:
import numpa
import numpy as np
import pandas as pd
import pyarrow.parquet as pq

def load_df(path, columns=None, nthreads=4, strings_to_categorical=True):
    try:
        table = pq.read_table(path, columns=columns, nthreads=nthreads)
        return table.to_pandas(strings_to_categorical=strings_to_categorical)
    except Exception as e:
        print(e)

df = load_df('../src/combinations.parquet')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2854918 entries, 0 to 2854917
Data columns (total 7 columns):
device_id         uint16
timestamp         datetime64[ns]
battery_state     category
battery_level     uint8
network_status    category
time_diff         float64
combination       uint8
dtypes: category(2), datetime64[ns](1), float64(1), uint16(1), uint8(2)
memory usage: 81.7 MB


In [33]:
def truth_table(k):
    """
    Generate a truth table of dimension (k, k)
    """
    x = np.array([0, 1], dtype=np.int8)
    mesh = np.meshgrid(*([x] * k))
    return np.vstack([y.flat for y in mesh]).T


def pack_comb(haystack):
    """
    Return a decimal integer from a byte array
    """
    return np.packbits(np.array(haystack, dtype=np.uint8), axis=-1)


def unpack_comb(n):
    """
    Return a byte array from a decimal integer
    """
    return np.unpackbits(np.array([n], dtype=np.uint8))


@numba.jit
def apply_filter(n):
    """
    Apply the filter to the given combination
    """
    a = unpack_comb(n)

    # Hardcoded expression
    return (
        a[1] == True
    )


cols = ['screen_on', 'bluetooth_enabled', 'location_enabled', 'power_saver_enabled',
        'nfc_enabled', 'unknown_sources', 'developer_mode', 'wifi_enabled']

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()